In [ ]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.0 MB/s eta 0:00:00


In [ ]:
# Load model directly
from transformers import AutoProcessor, AutoModelForCTC

In [ ]:
processor = AutoProcessor.from_pretrained("henilp105/wav2vec2-large-xls-r-300m-telugu-asr")
model = AutoModelForCTC.from_pretrained("henilp105/wav2vec2-large-xls-r-300m-telugu-asr")

In [ ]:
# prepare the audio data into numbers
from datasets import load_dataset, Value

# Define a custom dataset
custom_dataset = load_dataset(
    'tsv',
    data_files={'train': '/content/train.tsv', 'test': '/content/test.tsv'},
    delimiter='\t',
    column_names=["audio_file_name", "transcript"],
    cache_dir='./dataset_cache'
)

# Define a custom audio processor
def process_audio(audio_path):
    # Add audio processing code here (e.g., converting audio to spectrograms, MFCCs, etc.)
    # Return processed audio features
    return processed_audio

# Map the processing function to the dataset
custom_dataset = custom_dataset.map(
    lambda example: {
        'input_values': process_audio(example['audio_file_name']),
        'transcript': example['transcript']
    },
    remove_columns=['audio_path']
)

# Define how to compute the targets
def compute_targets(example):
    # Convert transcript to target tokens (e.g., using a tokenizer)
    # Return the target token IDs
    return target_token_ids

# Map the target computation function to the dataset
custom_dataset = custom_dataset.map(
    compute_targets,
    batched=True,
    remove_columns=['transcript'],
    fn_kwargs={'batch_size': 1}
)

# Set format and columns
custom_dataset.set_format(
    type='torch',
    columns=['input_values', 'attention_mask', 'labels']
)

# Print example
print(custom_dataset['train'][0])


In [ ]:
# import torch
# from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
# from datasets import load_dataset

# # Load the pretrained processor and model
# pretrained_model_name = "facebook/wav2vec2-base-960h"
# processor = Wav2Vec2Processor.from_pretrained(pretrained_model_name)
# model = Wav2Vec2ForCTC.from_pretrained(pretrained_model_name)

# # Load and preprocess the ASR dataset
# dataset = load_dataset("common_voice", "en", split="train")
# # Assuming dataset contains audio files and corresponding text transcripts

# # Define data processing and collation functions
# def prepare_dataset(batch):
#     inputs = processor(batch["speech"], sampling_rate=16000, return_tensors="pt").input_values
#     targets = processor.batch_encode_plus(batch["sentence"], return_tensors="pt", padding=True).input_ids
#     return {"input_values": inputs, "labels": targets}

# # Prepare the dataset for fine-tuning
# dataset = dataset.map(prepare_dataset, batched=True)

# # Fine-tuning hyperparameters
# num_epochs = 10
# batch_size = 4
# learning_rate = 1e-4

# # Initialize optimizer and loss function
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
# loss_fn = torch.nn.CTCLoss(blank=model.config.pad_token_id)

# # Fine-tuning loop
# for epoch in range(num_epochs):
#     dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
#     total_loss = 0.0

#     for batch in dataloader:
#         inputs = batch["input_values"]
#         labels = batch["labels"]

#         optimizer.zero_grad()
#         logits = model(input_values=inputs).logits

#         input_lengths = torch.full(size=(batch_size,), fill_value=logits.shape[1], dtype=torch.int32)
#         label_lengths = torch.tensor([len(label) for label in labels], dtype=torch.int32)

#         loss = loss_fn(logits, labels, input_lengths, label_lengths)
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#     print(f"Epoch {epoch+1}: Average Loss: {total_loss / len(dataloader)}")

# # Save the fine-tuned model
# model.save_pretrained("fine_tuned_asr_model")
